In [ ]:
pip install espn_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for espn-api: filename=espn_api-0.28.0-py3-none-any.whl size=58601 sha256=07490c1e389adfac7b6f203579d2818184332bc61d0814e4512b1bc339a4464c
  Stored in directory: /root/.cache/pip/wheels/31/c3/a6/ce391ed05e75a5212393e8b22d7503a05bf1b7b4e84ee3444b
Successfully built espn-api


In [ ]:
# Import Basketball API and all relevant libraries
from espn_api.basketball import League
from espn_api.basketball import Player

import pandas as pd
import numpy as np
import math

import time
import datetime as dt
from dateutil import tz

# Setup the league variables for this year and previous year.
leaguePreviousYear = League(league_id=1483720569, year=2022, espn_s2='AEAcEt%2BMhB7Fge2Z8ytHMzkMCzI3Pv9P2iScn2FG4%2BjItbmIma%2BKoAP32crD2jneK0ZSIpkCeuMewE2tWpBSh8V427kylczAMaNqUpWA3XlpILa8IUyjLY%2BiKLeNdEobg0TBvOfYzWIxEJYdE5%2FctLuNNvV9GVpzc4u2I6FTdEleJgkb4FLU1NGuK8CAWppO8eI0cYgIWpWfNfFHIw9NmlqFTH57vdXgYRGNq3pGJuLZxgVIq54ghOwrsynfR8uQ2yjhAwI7n74p5eqgMJ9Ifnh74mvMGOFeArbTx9KMKkYeWg%3D%3D', 
                swid='{AA6619A1-D92E-4220-BECC-9A41FA2A0B0D}')
league = League(league_id=1483720569, year=2023, espn_s2='AEAcEt%2BMhB7Fge2Z8ytHMzkMCzI3Pv9P2iScn2FG4%2BjItbmIma%2BKoAP32crD2jneK0ZSIpkCeuMewE2tWpBSh8V427kylczAMaNqUpWA3XlpILa8IUyjLY%2BiKLeNdEobg0TBvOfYzWIxEJYdE5%2FctLuNNvV9GVpzc4u2I6FTdEleJgkb4FLU1NGuK8CAWppO8eI0cYgIWpWfNfFHIw9NmlqFTH57vdXgYRGNq3pGJuLZxgVIq54ghOwrsynfR8uQ2yjhAwI7n74p5eqgMJ9Ifnh74mvMGOFeArbTx9KMKkYeWg%3D%3D', 
                swid='{AA6619A1-D92E-4220-BECC-9A41FA2A0B0D}')

## Constants

In [ ]:
#Constants
dayZero = dt.datetime(2022,10,16).timestamp()
dayZerodt = dt.date(2022,10,16)

## Getter Functions

In [ ]:
#Base level outside of API/ helper functions

def getCurrentWeekNum():
  today = dt.datetime.today().timestamp()
  weeks = (today - dayZero) / 604800
  weekNum = math.ceil(weeks)
  return weekNum

def getLastWeekNum():
  return getCurrentWeekNum() - 1

def createPlayerDataFrame(player):
  playerStats = pd.DataFrame(player.stats).transpose()
  playerStatsIndex = playerStats.index
  playerStats['name'] = player.name
  playerStats = playerStats.set_index(['name',playerStatsIndex])
  return playerStats


#Base level of API, but return all functions as dataframes for easier manipulation. 
#The functions will return API objects, and so they will need to be transformed appropriately in whatever function they are later used in.

#Get all teams not in ranked order
def getTeams():
  return pd.DataFrame(league.teams)

#Get standings
def getStandings():
  return pd.DataFrame(league.standings())

#Get Recent Activity for activity
def getRecentActivity(size, activity):
  return pd.DataFrame(league.recent_activity(size=size,msg_type=activity))

#Get all free agents
def getFreeAgents():
  return pd.DataFrame(league.free_agents(size=1000))

#Get scoreboard for weekNum week
def getScoreboard(weekNum):
  return pd.DataFrame(league.scoreboard(weekNum))

#Get all boxscores for weekNum week
def getBoxScores(weekNum):
  return pd.DataFrame(league.box_scores(weekNum))

#Get all boxscores for weekNum week
def getBoxScores_adv(scoring_period, matchup_total):
  return pd.DataFrame(league.box_scores(scoring_period=scoring_period, matchup_total=matchup_total))

#Retrieve saved standings from last week.
def getSavedStandings():
  weekNum = getCurrentWeekNum()
  with open('/content/drive/MyDrive/Personal Projects/NBA_Fantasy/Standings - Week of {0}.txt'.format(dayZerodt + dt.timedelta(weeks=weekNum-1)), 'r') as f:
    standings = f.read().splitlines()
    standings = pd.DataFrame(standings)
    standings.index += 1
    standings.rename(columns={0:"Team"}, inplace=True)
    standings['Rank'] = standings.index
  return standings

# Advanced getter functions, using and transforming base getter functions.

#Get rankings
def getRankings():
  standings = getStandings()
  standings.index += 1
  standings.rename(columns={0:"Team"}, inplace=True)
  standings['Rank'] = standings.index
  standings['Team'] = standings.apply(lambda row: row['Team'].team_name, axis=1)
  return standings

#Get all players on rosters
def getRosterPlayers():
  teams = getTeams()
  players = teams.apply(lambda team: team[0].roster, axis=1)
  players = players.explode()
  players = players.reset_index(drop=True)
  return players

#Get all players in the league. This will be all players on rosters + all free agents.
def getPlayers():
  return pd.concat([getRosterPlayers(), getFreeAgents()])

#Get a multi-indexed dataframe that contains the stats of all players
def getPlayerStats():
  players = getPlayers()
  playerStats = []
  for row in players.itertuples():
    playerStats.append(createPlayerDataFrame(row._1))
  playerStats = pd.concat(playerStats)
  return playerStats

def getFAstats():
  players = getFreeAgents()
  playerStats = []
  for row in players.itertuples():
    playerStats.append(createPlayerDataFrame(row._1))
  FAstats = pd.concat(playerStats)
  return FAstats

def getBoxPlayer(scoringPeriod):
  boxscores = getBoxScores_adv(scoringPeriod, False)
  boxscores['HomeBoxLineup'] = boxscores.apply(lambda matchup: matchup[0].home_lineup, axis=1)
  boxscores['AwayBoxLineup'] = boxscores.apply(lambda matchup: matchup[0].away_lineup, axis=1)
  BoxPlayer = pd.concat([boxscores['HomeBoxLineup'],boxscores['AwayBoxLineup']])
  BoxPlayer = BoxPlayer.explode().reset_index()
  BoxPlayer['Player'] = BoxPlayer.apply(lambda boxplayer: boxplayer[0].name, axis=1)
  BoxPlayer['Points'] = BoxPlayer.apply(lambda boxplayer: boxplayer[0].points, axis=1)
  BoxPlayer.rename(columns={0:"Tuple"}, inplace=True)
  return BoxPlayer

#This works in colab, but not Power Bi
def getBestPlayers():
  playerStats = getPlayerStats()
  playerStats = playerStats.drop('2023_total', level=1, axis=0)
  playerStats = playerStats.drop('2023_last_15', level=1, axis=0)
  playerStats = playerStats.drop('2022_total', level=1, axis=0)
  playerStats = playerStats.drop('2023_last_30', level=1, axis=0)
  playerStats = playerStats.drop('2023_projected', level=1, axis=0)
  playerStats = playerStats.drop('2022_projected', level=1, axis=0)
  playerStats = playerStats.droplevel(1)
  playerStats['Name'] = playerStats.index
  return playerStats

#Works in power Bi
def getBestPlayersLastWeek():
  playerStats = getPlayerStats()
  playerStats['Tuple'] = playerStats.index.to_numpy()
  playerStats['Name'] = playerStats.apply(lambda row: row['Tuple'][0], axis=1)
  playerStats['Stat'] = playerStats.apply(lambda row: row['Tuple'][1], axis=1)
  playerStats = playerStats[playerStats['Stat']=='2023_last_7']
  return playerStats

def getBestFreeAgentsLastWeek():
  faStats = getFAstats()
  faStats['Tuple'] = faStats.index.to_numpy()
  faStats['Name'] = faStats.apply(lambda row: row['Tuple'][0], axis=1)
  faStats['Stat'] = faStats.apply(lambda row: row['Tuple'][1], axis=1)
  faStats = faStats[faStats['Stat']=='2023_last_7']
  return faStats

#Used in getPickupScores() to calculate the total FPTS scored in a week for a player.
def getTotalScoredByPickup(row):
  playername = row['Player']
  pickupDate = row['Date (epoch ms)']
  pickupPeriod = ((pickupDate / 1000) - dayZero) / 86400
  dfPlayerArray = []
  if (pickupPeriod % 1) > 0.71:
    startPeriod = math.ceil(pickupPeriod)
  else:
    startPeriod = math.floor(pickupPeriod)
  endPeriod = int(dt.timedelta(weeks=getLastWeekNum()).total_seconds() / 86400) #Get's last day of the week, and computes number of days since dayZero
  for period in range(startPeriod, endPeriod+1):
    if (period != 23) and (period != 39):
      boxplayers = getBoxPlayer(period)
      boxPlayer = boxplayers[boxplayers['Player']==playername]
      dfPlayerArray.append(boxPlayer)
  if dfPlayerArray != []:
    dfPlayer = pd.concat(dfPlayerArray)
    points = np.sum(dfPlayer['Points'])
  else:
    points = 0
  return points


#Used in getPickupScores() to calculate the average FPTS scored in a week for a player.
def getAverageScoredByPickup(row):
  playername = row['Player']
  pickupDate = row['Date (epoch ms)']
  pickupPeriod = ((pickupDate / 1000) - dayZero) / 86400
  dfPlayerArray = []
  if (pickupPeriod % 1) > 0.71:
    startPeriod = math.ceil(pickupPeriod)
  else:
    startPeriod = math.floor(pickupPeriod)
  endPeriod = int(dt.timedelta(weeks=getLastWeekNum()).total_seconds() / 86400) #Get's last day of the week, and computes number of days since dayZero
  for period in range(startPeriod, endPeriod+1):
    if (period != 23) and (period != 39):
      boxplayers = getBoxPlayer(period)
      boxPlayer = boxplayers[boxplayers['Player']==playername]
      dfPlayerArray.append(boxPlayer)
  if dfPlayerArray != []:
    dfPlayer = pd.concat(dfPlayerArray)
    points = np.sum(dfPlayer['Points'])
    dfPlayer['gamePlayed'] = dfPlayer.apply(lambda row: int(row['Points'] != 0), axis=1)
    gamesPlayed = np.sum(dfPlayer['gamePlayed'])

    if gamesPlayed != 0:
      avgPoints = points / gamesPlayed
    else:
      avgPoints = 0

  else:
    avgPoints = 0

  return avgPoints

#Get total fantasy points scored
def getTotalFPTS(weekNum):
  allScoreboards = []
  for week in range(weekNum):
    scoreboard = getScoreboard(week)
    scoreboard['AwayTeam'] = scoreboard.apply(lambda matchup: matchup[0].away_team, axis=1)
    scoreboard['AwayScore'] = scoreboard.apply(lambda matchup: matchup[0].away_final_score, axis=1) 
    scoreboard['HomeTeam'] = scoreboard.apply(lambda matchup: matchup[0].home_team, axis=1)
    scoreboard['HomeScore'] = scoreboard.apply(lambda matchup: matchup[0].home_final_score, axis=1)
    scoreboard_away = scoreboard[['AwayTeam', 'AwayScore']].rename(columns={'AwayTeam':'Team', 'AwayScore':week})
    scoreboard_home = scoreboard[['HomeTeam', 'HomeScore']].rename(columns={'HomeTeam':'Team', 'HomeScore':week})

    Scoreboard = pd.concat([scoreboard_away, scoreboard_home], axis=0).set_index('Team', drop=True)

    allScoreboards.append(Scoreboard)

  totalFPTS = pd.concat(allScoreboards, axis=1)
  totalFPTS['Total'] = totalFPTS.apply(lambda x: sum(x), axis=1)
  totalFPTS.index = totalFPTS.index.map(lambda team: team.team_name)
  totalFPTS = totalFPTS.reset_index() 

  return totalFPTS

#Get closest matchups from last week
def getCloseMatchups(weekNum):
  scoreboard = getScoreboard(weekNum)
  scoreboard['AwayTeam'] = scoreboard.apply(lambda matchup: matchup[0].away_team.team_name, axis=1)
  scoreboard['AwayScore'] = scoreboard.apply(lambda matchup: matchup[0].away_final_score, axis=1) 
  scoreboard['HomeTeam'] = scoreboard.apply(lambda matchup: matchup[0].home_team.team_name, axis=1)
  scoreboard['HomeScore'] = scoreboard.apply(lambda matchup: matchup[0].home_final_score, axis=1)
  scoreboard['Difference'] = np.abs(scoreboard['AwayScore'] - scoreboard['HomeScore'])

  closeMatchups = scoreboard.query("Difference < 30")
  return closeMatchups

#Get upcoming power matchups for this week.
def getPowerMatchups(weekNum):
  scoreboard = getScoreboard(weekNum)
  scoreboard['AwayTeam'] = scoreboard.apply(lambda matchup: matchup[0].away_team.team_name, axis=1)
  scoreboard['AwayRank'] = scoreboard.apply(lambda matchup: matchup[0].away_team.standing, axis=1)
  scoreboard['HomeTeam'] = scoreboard.apply(lambda matchup: matchup[0].home_team.team_name, axis=1)
  scoreboard['HomeRank'] = scoreboard.apply(lambda matchup: matchup[0].home_team.standing, axis=1)
  scoreboard['Strength'] = np.abs(scoreboard['AwayRank'] - scoreboard['HomeRank']) + scoreboard['AwayRank']
  
  powerMatchups = scoreboard.query("Strength < 5")
  return powerMatchups

#Get underdog victories
def getUnderdogWins(weekNum):
  scoreboard = getScoreboard(weekNum)
  scoreboard['AwayTeam'] = scoreboard.apply(lambda matchup: matchup[0].away_team.team_name, axis=1)
  scoreboard['AwayRank'] = scoreboard.apply(lambda matchup: matchup[0].away_team.standing, axis=1)
  scoreboard['HomeTeam'] = scoreboard.apply(lambda matchup: matchup[0].home_team.team_name, axis=1)
  scoreboard['HomeRank'] = scoreboard.apply(lambda matchup: matchup[0].home_team.standing, axis=1)
  scoreboard['Winner'] = scoreboard.apply(lambda matchup: matchup[0].winner, axis=1)
  scoreboard['WinnerName'] = scoreboard.apply(lambda row: row['AwayTeam'] if row['Winner']=='AWAY' else row['HomeTeam'], axis=1)
  scoreboard['Rank Difference'] = scoreboard['AwayRank'] - scoreboard['HomeRank']
  scoreboard['UnderdogWin'] = scoreboard.apply(lambda row: (row['Rank Difference'] > 0 and row['Winner'] == 'AWAY') or row['Rank Difference'] < 0 and row['Winner'] == 'HOME', axis=1)
  scoreboard = scoreboard.query('UnderdogWin == True')
  
  return scoreboard


#Wins where the winning team was down before the start of the last day.
def getComebackWins():
  lastWeekNum = getLastWeekNum() 
  startPeriod = (lastWeekNum * 7) - 6
  endPeriod = lastWeekNum * 7 + 1
  allBoxScores = []
  for period in range(startPeriod, endPeriod):
    boxscore = getBoxScores_adv(scoring_period=period, matchup_total=False)
    boxscore['AwayTeam'] = boxscore.apply(lambda matchup: matchup[0].away_team, axis=1)
    boxscore['AwayScore'] = boxscore.apply(lambda matchup: matchup[0].away_score, axis=1) 
    boxscore['HomeTeam'] = boxscore.apply(lambda matchup: matchup[0].home_team, axis=1)
    boxscore['HomeScore'] = boxscore.apply(lambda matchup: matchup[0].home_score, axis=1)
    boxscore_away = boxscore[['AwayTeam', 'AwayScore']].rename(columns={'AwayTeam':'Team', 'AwayScore':period-startPeriod+1})
    boxscore_home = boxscore[['HomeTeam', 'HomeScore']].rename(columns={'HomeTeam':'Team', 'HomeScore':period-startPeriod+1})
    boxscores = pd.concat([boxscore_away, boxscore_home], axis=0).set_index('Team', drop=True)

    allBoxScores.append(boxscores)

  totalscore = pd.concat(allBoxScores, axis=1)
  totalscore['Total'] = totalscore.apply(lambda x: sum(x), axis=1)
  totalscore['Last Day'] = totalscore.apply(lambda x: sum(x[0:6]),axis=1)
  totalscore.index = totalscore.index.map(lambda team: team.team_name)

  #Reconstruct matchups and compare winner to last day scores
  matchupInfo = getBoxScores(lastWeekNum)
  matchupInfo['AwayTeam'] = matchupInfo.apply(lambda matchup: matchup[0].away_team.team_name, axis=1)
  matchupInfo['AwayFinalScore'] = matchupInfo.apply(lambda matchup: matchup[0].away_score, axis=1)
  matchupInfo['HomeTeam'] = matchupInfo.apply(lambda matchup: matchup[0].home_team.team_name, axis=1)
  matchupInfo['HomeFinalScore'] = matchupInfo.apply(lambda matchup: matchup[0].home_score, axis=1)
  matchupInfo['AwayLastScore'] = matchupInfo.apply(lambda row: totalscore['Last Day'].loc[row['AwayTeam']], axis=1)
  matchupInfo['HomeLastScore'] = matchupInfo.apply(lambda row: totalscore['Last Day'].loc[row['HomeTeam']], axis=1)
  matchupInfo['WINNER'] = matchupInfo.apply(lambda matchup: matchup[0].home_team.team_name if matchup[0].winner=='HOME' else matchup[0].away_team.team_name, axis=1)
  matchupInfo = matchupInfo[['AwayTeam', 'AwayFinalScore', 'AwayLastScore', 'HomeTeam', 'HomeFinalScore', 'HomeLastScore', 'WINNER']]

  comebackWins = matchupInfo.query('(AwayFinalScore > HomeFinalScore & AwayLastScore < HomeLastScore) | (HomeFinalScore > AwayFinalScore & HomeLastScore < AwayLastScore)')

  return comebackWins

#Get free agents that were added during the week and are still on the roster at the end of the week, and calculate their total and average scores for the week.
def getPickupScores():
  starttime = (dayZero + dt.timedelta(hours=6,days=1,weeks=getLastWeekNum()-1).total_seconds()) * 1000
  endtime = (dayZero + dt.timedelta(days=1,weeks=getLastWeekNum()).total_seconds()) * 1000

  recentActivity = getRecentActivity(60, 'FA')
  recentActivity['Team'] = recentActivity.apply(lambda activity: activity[0].actions[0][0], axis=1)
  recentActivity['Activity'] = recentActivity.apply(lambda activity: activity[0].actions[0][1], axis=1)
  recentActivity['Player'] = recentActivity.apply(lambda activity: activity[0].actions[0][2], axis=1)
  recentActivity['Date (epoch ms)'] = recentActivity.apply(lambda activity: activity[0].date, axis=1)
  recentActivity['Roster'] = recentActivity.apply(lambda activity: [player.name for player in activity['Team'].roster],axis=1)
  recentActivity['OnRoster'] = recentActivity.apply(lambda activity: activity['Player'] in activity['Roster'], axis=1)

  recentActivity_waiver = getRecentActivity(60, 'WAIVER')
  recentActivity_waiver['Team'] = recentActivity_waiver.apply(lambda activity: activity[0].actions[0][0], axis=1)
  recentActivity_waiver['Activity'] = recentActivity_waiver.apply(lambda activity: activity[0].actions[0][1], axis=1)
  recentActivity_waiver['Player'] = recentActivity_waiver.apply(lambda activity: activity[0].actions[0][2], axis=1)
  recentActivity_waiver['Date (epoch ms)'] = recentActivity_waiver.apply(lambda activity: activity[0].date, axis=1)
  recentActivity_waiver['Roster'] = recentActivity_waiver.apply(lambda activity: [player.name for player in activity['Team'].roster],axis=1)
  recentActivity_waiver['OnRoster'] = recentActivity_waiver.apply(lambda activity: activity['Player'] in activity['Roster'], axis=1)

  recentActivity = pd.concat([recentActivity,recentActivity_waiver], axis=0)
  recentActivity = recentActivity.sort_values('Date (epoch ms)', ascending=False)
  recentActivity['TeamName'] = recentActivity.apply(lambda row: row['Team'].team_name, axis=1)

  recentActivity = recentActivity[recentActivity['Date (epoch ms)'] <= endtime]
  recentActivity = recentActivity[recentActivity['Date (epoch ms)'] >= starttime]
  recentActivity = recentActivity.query('OnRoster == True')
  recentActivity['totalPoints'] = recentActivity.apply(lambda row: getTotalScoredByPickup(row), axis=1)
  recentActivity['avgPoints'] = recentActivity.apply(lambda row: getAverageScoredByPickup(row), axis=1)

  return recentActivity

In [ ]:
getBestFreeAgentsLastWeek()

,,applied_total,applied_avg,team,date,avg,total,Tuple,Name,Stat
name,,,,,,,,,,
Saddiq Bey,2023_last_7,25.0,25.0,None,None,"{'PTS': 15.0, 'BLK': 1.0, 'STL': 2.0, 'AST': 0...","{'PTS': 15.0, 'BLK': 1.0, 'STL': 2.0, 'AST': 0...","(Saddiq Bey, 2023_last_7)",Saddiq Bey,2023_last_7
Keegan Murray,2023_last_7,16.0,5.33,None,None,"{'PTS': 3.0, 'BLK': 0.3333333333333333, 'STL':...","{'PTS': 9.0, 'BLK': 1.0, 'STL': 1.0, 'AST': 2....","(Keegan Murray, 2023_last_7)",Keegan Murray,2023_last_7
Gordon Hayward,2023_last_7,4.0,4.0,None,None,"{'PTS': 7.0, 'BLK': 0.0, 'STL': 0.0, 'AST': 3....","{'PTS': 7.0, 'BLK': 0.0, 'STL': 0.0, 'AST': 3....","(Gordon Hayward, 2023_last_7)",Gordon Hayward,2023_last_7
John Wall,2023_last_7,85.0,21.25,None,None,"{'PTS': 13.75, 'BLK': 0.25, 'STL': 0.25, 'AST'...","{'PTS': 55.0, 'BLK': 1.0, 'STL': 1.0, 'AST': 2...","(John Wall, 2023_last_7)",John Wall,2023_last_7
Brandon Clarke,2023_last_7,64.0,21.33,None,None,"{'PTS': 9.0, 'BLK': 1.0, 'STL': 0.333333333333...","{'PTS': 27.0, 'BLK': 3.0, 'STL': 1.0, 'AST': 3...","(Brandon Clarke, 2023_last_7)",Brandon Clarke,2023_last_7
...,...,...,...,...,...,...,...,...,...,...
Ashton Hagans,2023_last_7,0.0,0.0,NaN,NaN,NaN,NaN,"(Ashton Hagans, 2023_last_7)",Ashton Hagans,2023_last_7
Rawle Alkins,2023_last_7,0.0,0.0,NaN,NaN,NaN,NaN,"(Rawle Alkins, 2023_last_7)",Rawle Alkins,2023_last_7
Isaia Cordinier,2023_last_7,0.0,0.0,NaN,NaN,NaN,NaN,"(Isaia Cordinier, 2023_last_7)",Isaia Cordinier,2023_last_7


In [ ]:
playername = "De'Anthony Melton"
pickupDate = 1669258148094	
pickupPeriod = ((pickupDate / 1000) - dayZero) / 86400
dfPlayerArray = []
if (pickupPeriod % 1) > 0.71:
  startPeriod = math.ceil(pickupPeriod)
else:
  startPeriod = math.floor(pickupPeriod)
endPeriod = int(dt.timedelta(weeks=getLastWeekNum()).total_seconds() / 86400) #Get's last day of the week, and computes number of days since dayZero
for period in range(startPeriod, endPeriod+1):
  if (period != 23) and (period != 39):
    boxplayers = getBoxPlayer(period)
    boxPlayer = boxplayers[boxplayers['Player']==playername]
    dfPlayerArray.append(boxPlayer)
if dfPlayerArray != []:
  dfPlayer = pd.concat(dfPlayerArray)
  points = np.sum(dfPlayer['Points'])
else:
  points = 0
points

59.0

In [ ]:
pickupscores = getPickupScores()
pickupscores

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:314: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,0,Team,Activity,Player,Date (epoch ms),Roster,OnRoster,TeamName,totalPoints,avgPoints
2,"Activity((Team(Big Gabes),FA ADDED,Collin Sex...",Team(Big Gabes),FA ADDED,Collin Sexton,1669579551059,"[Luka Doncic, DeMar DeRozan, Dejounte Murray, ...",True,Big Gabes,0.0,0.000000
4,"Activity((Team(House Yarish),FA ADDED,Bennedic...",Team(House Yarish),FA ADDED,Bennedict Mathurin,1669519857112,"[Jayson Tatum, Tyrese Haliburton, Darius Garla...",True,House Yarish,14.0,14.000000
7,"Activity((Team(House Huynh),FA ADDED,Jabari Sm...",Team(House Huynh),FA ADDED,Jabari Smith Jr.,1669469954619,"[James Harden, Pascal Siakam, Zion Williamson,...",True,House Huynh,32.0,32.000000
9,"Activity((Team(Powie wowie Ballers),FA ADDED,M...",Team(Powie wowie Ballers),FA ADDED,Marvin Bagley III,1669369276044,"[LeBron James, LaMelo Ball, Kristaps Porzingis...",True,Powie wowie Ballers,72.0,36.000000
11,"Activity((Team(House Yarish),FA ADDED,De'Antho...",Team(House Yarish),FA ADDED,De'Anthony Melton,1669258148094,"[Jayson Tatum, Tyrese Haliburton, Darius Garla...",True,House Yarish,59.0,29.500000
12,"Activity((Team(Chris’s Team Ded Inside),FA ADD...",Team(Chris’s Team Ded Inside),FA ADDED,Chris Boucher,1669245204873,"[Giannis Antetokounmpo, Devin Booker, Kyrie Ir...",True,Chris’s Team Ded Inside,37.0,37.000000
13,"Activity((Team(Grand blue Straw hats),FA ADDED...",Team(Grand blue Straw hats),FA ADDED,Reggie Jackson,1669144694930,"[Ja Morant, Scottie Barnes, Terry Rozier, Jimm...",True,Grand blue Straw hats,57.0,19.000000
14,"Activity((Team(House Huynh),FA ADDED,Klay Thom...",Team(House Huynh),FA ADDED,Klay Thompson,1669046752012,"[James Harden, Pascal Siakam, Zion Williamson,...",True,House Huynh,95.0,31.666667
0,"Activity((Team(Big Gabes),WAIVER ADDED,Jalen ...",Team(Big Gabes),WAIVER ADDED,Jalen Suggs,1669022791112,"[Luka Doncic, DeMar DeRozan, Dejounte Murray, ...",True,Big Gabes,38.0,19.000000


## Setter Functions

In [ ]:
#Save current standings
def saveStandings():
  standings = getStandings()
  weekNum = getLastWeekNum()
  with open('/content/drive/MyDrive/Personal Projects/NBA_Fantasy/Standings - Week of {0}.txt'.format(dayZerodt + dt.timedelta(weeks=weekNum)), 'w') as f:
    for row in standings.itertuples():
      f.write(row[1].team_name)
      f.write('\n')

## OLD SCRIPTS BELOW


In [ ]:
## FINISHED
#Standings
#Last week's closest matchup
#Best performing player of the week
#Best performing player not on a team
#Upcoming power matchups
#Power matchup results from last week

## NOT FINISHED
#Best Free agent pickup

In [ ]:
# Helper functions

#First day of NBA fantasy on ESPN begins Oct 17, 2022 (This is scoring_period=1). The first sunday (scoring_period=0) is Oct 16, 2022.
firstSunday = dt.datetime(2022,10,16).timestamp()

#Get previous week number
def getCurrentWeekNum():
  firstSunday = dt.datetime(2022,10,16).timestamp()
  today = dt.datetime.today().timestamp()
  weeks = (today - firstSunday) / 604800
  weekNum = math.ceil(weeks)
  return weekNum

def getLastWeekNum():
  return getCurrentWeekNum() - 1

#Get scoring periods
def getScoringPeriods(startDate):
  startDate = (startDate / 1000) - 5*3600
  firstSunday = dt.datetime(2022,10,16).timestamp()
  #today = dt.datetime.today().timestamp()
  #endPeriod = ((today - firstSunday) - 2*86400) / 86400
  endPeriod = (dt.timedelta(weeks=getLastWeekNum())).total_seconds() / 86400
  endPeriod = round(endPeriod) 
  startPeriod = (startDate - firstSunday) / 86400
  startPeriod = round(startPeriod)

  return startPeriod, endPeriod


In [ ]:
#Save current standings
def saveStandings():
  with open('/content/drive/MyDrive/Personal Projects/NBA_Fantasy/Standings - Week of {0}.txt'.format(dt.date.today()), 'w') as f:
    for team in league.standings():
      f.write(team.team_name)
      f.write('\n')

#Retrieve saved standings from last week.
def getSavedStandings():
  with open('/content/drive/MyDrive/Personal Projects/NBA_Fantasy/Standings - Week of {0}.txt'.format(dt.date.today()-dt.timedelta(days=7)), 'r') as f:
    standings = f.read().splitlines()
    standings = pd.DataFrame(standings)
    standings.index += 1
    standings.rename(columns={0:"Team"}, inplace=True)
    standings['Rank'] = standings.index
  return standings

In [ ]:
#Standings
#MONDAY MORNING
def getStandings():
  standings = pd.DataFrame(league.standings())
  standings.index += 1
  standings.rename(columns={0:"Team"}, inplace=True)
  standings['Team'] = standings.apply(lambda x: x['Team'].team_name, axis=1)
  standings['Rank'] = standings.index
  return standings

#Last week's closest matchup
#MONDAY MORNING

def getClosestMatchup():
  weekNum = getLastWeekNum()
  matchups = league.box_scores(weekNum)

  scoreDifferences = list(map(lambda x: abs(x.home_score - x.away_score), matchups))
  matchupScores = zip(matchups, scoreDifferences)
  closeMatchupScores = sorted(matchupScores, key = lambda x: x[1])

  closestMatchups = [matchup for matchup in closeMatchupScores if abs(matchup[1] - closeMatchupScores[0][1]) < 30]

  closestMatchupsdf = pd.DataFrame(closestMatchups)
  closestMatchupsdf['AWAY'] = closestMatchupsdf.apply(lambda x: x[0].away_team.team_name, axis=1)
  closestMatchupsdf['HOME'] = closestMatchupsdf.apply(lambda x: x[0].home_team.team_name, axis=1) 
  closestMatchupsdf['WINNER'] = closestMatchupsdf.apply(lambda x: x[0].away_team.team_name if x[0].winner == "AWAY" else x[0].home_team.team_name, axis=1)
  closestMatchupsdf = closestMatchupsdf.rename(columns = {0: "Boxscore Object", 1: "Difference"})

  return closestMatchupsdf

#Best perfoming player of the week on a team
#MONDAY MORNING

def getBestPlayersInLineups():
  teams = league.teams
  playersOnRosters = [(player, team.team_name) for team in teams for player in team.roster]

  bestPlayersInLineups = [(player[0].stats['2023_last_7']['applied_total'], player[0].name, player[1]) for player in playersOnRosters]
  bestPlayersInLineups = sorted(bestPlayersInLineups, key = lambda x: x[0], reverse=True)
  bestPlayersInLineups = bestPlayersInLineups[:5]

  bestPlayersdf = pd.DataFrame(bestPlayersInLineups)
  bestPlayersdf = bestPlayersdf.rename(columns={0:"Total Score", 1:"Player", 2:"Team"})

  return bestPlayersdf

#Best performing player not on a team
#MONDAY MORNING
def getBestFreeAgents():
  freeAgents = league.free_agents(size=500)

  bestFreeAgents = [(fa.stats['2023_last_7']['applied_total'], fa.name) for fa in freeAgents if (fa.projected_avg_points) != 0]
  bestFreeAgents = sorted(bestFreeAgents, key = lambda x: x[0], reverse=True)
  bestFreeAgents = bestFreeAgents[:5]

  bestFreeAgentsdf = pd.DataFrame(bestFreeAgents)
  bestFreeAgentsdf = bestFreeAgentsdf.rename(columns={0:"Total Score", 1:"Player"})

  return bestFreeAgentsdf

#Upcoming power matchups and previous power matchup results
#MONDAY MORNING

def rankSum(standings, matchup):
  sum = standings[standings['Team']==matchup.home_team.team_name]['Rank'].iloc[0] + standings[standings['Team']==matchup.away_team.team_name]['Rank'].iloc[0]
  return sum

def getPowerMatchups(prev):
  if prev == True:
    standings = getSavedStandings()
    weekNum = getLastWeekNum()
  else:
    standings = getStandings()  
    weekNum = getCurrentWeekNum()
  matchups = league.box_scores(weekNum)

  rankSums = list(map(lambda x: rankSum(standings, x), matchups))
  rankSums = list(enumerate(rankSums))
  rankSums = sorted(rankSums, key = lambda x: x[1])
  bestMatchups = [rank for rank in rankSums if rank[1]==rankSums[0][1]]
  bestMatchups = [matchups[matchup[0]] for matchup in bestMatchups]

  bestMatchupsdf = pd.DataFrame(bestMatchups)
  bestMatchupsdf['AWAY'] = bestMatchupsdf.apply(lambda x: x[0].away_team.team_name, axis=1)
  bestMatchupsdf['AWAY RANK'] = bestMatchupsdf.apply(lambda x: standings[standings['Team']==x[0].away_team.team_name]['Rank'].iloc[0], axis=1)
  bestMatchupsdf['HOME'] = bestMatchupsdf.apply(lambda x: x[0].home_team.team_name, axis=1)
  bestMatchupsdf['HOME RANK'] = bestMatchupsdf.apply(lambda x: standings[standings['Team']==x[0].home_team.team_name]['Rank'].iloc[0], axis=1) 

  return bestMatchupsdf

In [ ]:
#Best free agent pickup
#league.recent_activity()[0].actions[0][1]
def getLineups():
  teams = league.teams
  lineup = [player for team in teams for player in team.roster]
  return lineup

def getPickups():
  activities = league.recent_activity(msg_type = 'FA')
  pickups = [(activity.actions[0][2], activity.actions[0][0], activity.date) for activity in activities]
  return pickups

def pickupsStillOnTeams():
  pickups = getPickups()
  lineups = getLineups()
  pickupsOnTeams = [pickup for player in lineups for pickup in pickups if pickup[0]==player.name]
  return pickupsOnTeams

#date = dt.datetime.fromtimestamp(pickups[0][2] / 1000, dt.timezone(dt.timedelta(hours=-5)))
def getPickupScores():
  pickups = pickupsStillOnTeams()
  pickupsTotalScore = []
  for pickup in pickups:
    pickup = list(pickup)
    pickup.append(getScoringPeriods(pickup[2]))
    totalScore = 0
    gamesPlayed = 0
    scoringPeriodStart = pickup[3][0]
    scoringPeriodEnd = pickup[3][1]
    while(scoringPeriodEnd >= scoringPeriodStart):
      boxscores = league.box_scores(scoring_period=scoringPeriodStart, matchup_total=False)
      for boxscore in boxscores:
        if boxscore.away_team == pickup[1]:
          for player in boxscore.away_lineup:
            if (player.name == pickup[0]) == True:
              totalScore += player.points
              if player.points != 0:
                gamesPlayed += 1
        elif (boxscore.home_team == pickup[1]) == True:
          for player in boxscore.home_lineup:
            if player.name == pickup[0]:
              totalScore += player.points
              if player.points != 0:
                gamesPlayed += 1
      scoringPeriodStart += 1
    pickup.append(totalScore)
    if gamesPlayed != 0:
      pickup.append(totalScore / gamesPlayed)
    else:
      pickup.append(0)
    pickupsTotalScore.append(pickup)
  pickupsTotalScoredf = pd.DataFrame(pickupsTotalScore)
  pickupsTotalScoredf = pickupsTotalScoredf.rename(columns = {0:"Player", 1:"Team", 2:"Pickup Date (Epoch Seconds UTC)", 3:"Scoring Period Range", 4:"Total Points", 5:"Avg Points"})
  pickupsTotalScoredf['period diff'] = pickupsTotalScoredf.apply(lambda x: x['Scoring Period Range'][1] - x['Scoring Period Range'][0], axis=1)
  pickupsTotalScoredf = pickupsTotalScoredf[pickupsTotalScoredf['period diff'] >= 0]
  pickupsTotalScoredf['Team'] = pickupsTotalScoredf.apply(lambda x: x['Team'].team_name, axis=1)
  #pickupsTotalScoredf = pickupsTotalScoredf.drop()
  return pickupsTotalScoredf

In [ ]:
#New Functions to add:
#Underdog victory: matchup with large rank difference where lower ranked team won
#Win Streak: Display teams that are on a 2+ win streak
#Hospital Team Win: Teams that won with 2 or more players injured.

In [ ]:
#THESE FUNCTIONS DO NOT WORK IN POWER BI
def getPrevPowerMatchupResults():
  prevStandings = getSavedStandings()
  prevPowerMatchups = getPowerMatchups(True)
  prevPowerMatchups['WINNER'] = prevPowerMatchups.apply(lambda x: x[0].home_team.team_name if x[0].winner == 'HOME' else x[0].away_team.team_name, axis=1)

  return prevPowerMatchups

#Underdog victory:
def getUnderdogWins():
  standings = getSavedStandings()
  matchups = league.box_scores(getLastWeekNum())

  rankSums = list(map(lambda x: rankSum(standings, x), matchups))
  rankSums = list(enumerate(rankSums))
  rankSums = sorted(rankSums, key = lambda x: x[1], reverse=True)

  worstMatchups = rankSums[:2]
  worstMatchups = [(matchups[matchup[0]], (standings[standings['Team']==matchups[matchup[0]].away_team.team_name]['Rank'].iloc[0],standings[standings['Team']==matchups[matchup[0]].home_team.team_name]['Rank'].iloc[0])) for matchup in worstMatchups]

  underdogWins = []

  for matchup in worstMatchups:
    diff = matchup[1][1] - matchup[1][0]
    if diff > 0 and matchup[0].winner == 'HOME':
      underdogWins.append(matchup)
    if diff < 0 and matchup[0].winner == 'AWAY':
      underdogWins.append(matchup)

  underdogWins = pd.DataFrame(underdogWins)
  underdogWins['AWAY'] = underdogWins.apply(lambda x: x[0].away_team.team_name, axis=1)
  underdogWins['AWAY RANK'] = underdogWins.apply(lambda x: standings[standings['Team']==x[0].away_team.team_name]['Rank'].iloc[0], axis=1)
  underdogWins['HOME'] = underdogWins.apply(lambda x: x[0].home_team.team_name, axis=1)
  underdogWins['HOME RANK'] = underdogWins.apply(lambda x: standings[standings['Team']==x[0].home_team.team_name]['Rank'].iloc[0], axis=1)
  underdogWins['WINNER'] = underdogWins.apply(lambda x: x['AWAY'] if x[0].winner == 'AWAY' else x['HOME'], axis=1)
  
  return underdogWins
  

In [ ]:
underdogWins = getUnderdogWins()
prevPowerMatchupResults = getPrevPowerMatchupResults()
prevPowerMatchupResults

,0,AWAY,AWAY RANK,HOME,HOME RANK,WINNER
0,Box Score(Team(brooklyn park splashers) at Tea...,brooklyn park splashers,2,Team Big tings eh,3,brooklyn park splashers


In [ ]:
getCurrentWeekNum()

3